In [ ]:
from ezc3d import c3d
import pandas as pd
import numpy as np
from scipy.signal import butter, filtfilt

def process_c3d_data(file_name):    
    myc3d = c3d(file_name)

    # Create "full_df" DataFrame from the analog data
    analog_data = myc3d['data']['analogs']
    analog_labels = myc3d['parameters']['ANALOG']['LABELS']['value']
    analog_df = pd.DataFrame(data=analog_data[0, :, :], index=analog_labels)
    full_df = analog_df.T
    full_df.reset_index(inplace=True)
    full_df.rename(columns={'index': 'Time'}, inplace=True)

    print("All analog data columns:")
    # display(full_df)


    # Get linear force plate data ending with "Fx", "Fy", or "Fz"
    force_columns = [col for col in full_df.columns if col.endswith(("Fx", "Fy", "Fz"))]
    force_df = full_df[['Time'] + force_columns].copy()

    # Subtract the average value from each column to calibrate the force plate data
    average_values = force_df.iloc[:2000].mean()
    force_df.loc[:, force_df.columns != 'Time'] -= average_values.loc[force_df.columns != 'Time']

    contact_threshold = -10  # N
    force_df['contact'] = (force_df[force_columns] < contact_threshold).any(axis=1).astype(int)

    # print("Force plate data analysis: \n")

    first_contact_row = force_df['contact'].idxmax()
    takeoff_row = (force_df['contact'].diff() == -1).idxmax()
    second_contact_row = force_df.index[force_df['contact'].diff() == 1][1]
    ground_contact = takeoff_row - first_contact_row
    flight_time_ms = second_contact_row - takeoff_row
    jump_height = 9.81 * (flight_time_ms / 1000) ** 2 / 8
    rsi = jump_height / (ground_contact / 1000)

    # Get EMG data from CH1 and CH2 columns
    emg_df = full_df[['Time', 'CH1', 'CH2']].copy()

    # Define filter parameters
    lowcut = 50
    highcut = 400
    order = 2
    fs = 1000 # Hz Sampling frequency

    # Design band-pass Butterworth filter
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='band', analog=False)

    # Apply zero-lag filtering with filtfilt
    filtered_CH1 = filtfilt(b, a, emg_df['CH1'])
    filtered_CH2 = filtfilt(b, a, emg_df['CH2'])

    # Add filtered signals to DataFrame
    emg_df['CH1_filtered'] = filtered_CH1
    emg_df['CH2_filtered'] = filtered_CH2

    # Define RMS window size in samples (100 ms window)
    window_ms = 100
    window_size = int(fs * window_ms / 1000)

    # RMS function with rolling window
    def compute_rms(signal, window):
        squared = pd.Series(signal)**2
        mean_squared = squared.rolling(window, center=False).mean()  # no centering
        return np.sqrt(mean_squared)


    # Apply RMS
    emg_df['CH1_rms'] = compute_rms(emg_df['CH1_filtered'], window_size)
    emg_df['CH2_rms'] = compute_rms(emg_df['CH2_filtered'], window_size)

    # Determine trial height based on the file name
    trial_num = int(file_name.replace('c3d_files/Dynamic', '').replace('.c3d', ''))

    if 2 <= trial_num <= 11:
        trial_height = 0.3937
        drop_time = 283.31
    elif 12 <= trial_num <= 21:
        trial_height = 0.7366
        drop_time = 387.52
    elif 22 <= trial_num <= 31:
        trial_height = 0.9144
        drop_time = 431.77
    elif 32 <= trial_num <= 41:
        trial_height = 0.635
        drop_time = 359.81
    else:
        trial_height = None 
        drop_time = None

    est_step_off = int(first_contact_row - drop_time)

    # pre-activation
    max_ch1_rms_pre = emg_df.loc[est_step_off:first_contact_row, 'CH1_rms'].max()
    max_ch2_rms_pre = emg_df.loc[est_step_off:first_contact_row, 'CH2_rms'].max()

    max_ch1_rms_pre_row = emg_df.loc[est_step_off:first_contact_row, 'CH1_rms'].idxmax()
    max_ch2_rms_pre_row = emg_df.loc[est_step_off:first_contact_row, 'CH2_rms'].idxmax()

    ch1_preactivation = first_contact_row - max_ch1_rms_pre_row
    ch2_preactivation = first_contact_row - max_ch2_rms_pre_row

    # ground contact activation
    max_ch1_rms_ground = emg_df.loc[first_contact_row:takeoff_row, 'CH1_rms'].max()
    max_ch2_rms_ground = emg_df.loc[first_contact_row:takeoff_row, 'CH2_rms'].max()

    max_ch1_rms_ground_row = emg_df.loc[first_contact_row:takeoff_row, 'CH1_rms'].idxmax()
    max_ch2_rms_ground_row = emg_df.loc[first_contact_row:takeoff_row, 'CH2_rms'].idxmax()

    ch1_ground_activation = first_contact_row - max_ch1_rms_ground_row
    ch2_ground_activation = first_contact_row - max_ch2_rms_ground_row

    trial_output = {
        'first_contact_row': first_contact_row,
        'takeoff_row': takeoff_row,
        'second_contact_row': int(second_contact_row),
        'ground_contact': ground_contact,
        'flight_time_ms': int(flight_time_ms),
        'jump_height': float(jump_height),
        'rsi': float(rsi),
        'trial_height': trial_height,
        'drop_time': drop_time,
        'est_step_off': est_step_off,
        'max_ch1_rms_pre': float(max_ch1_rms_pre),
        'max_ch2_rms_pre': float(max_ch2_rms_pre),
        'max_ch1_rms_pre_row': max_ch1_rms_pre_row,
        'max_ch2_rms_pre_row': max_ch2_rms_pre_row,
        'ch1_preactivation': ch1_preactivation,
        'ch2_preactivation': ch2_preactivation
    }
    return trial_output

trial_heights = []
rsi_values = []
ch1_preactivation_values = []
ch2_preactivation_values = []


All analog data columns:


{'first_contact_row': 7002,
 'takeoff_row': 7654,
 'second_contact_row': 8715,
 'ground_contact': 652,
 'flight_time_ms': 1061,
 'jump_height': 1.38041537625,
 'rsi': 2.1172014973159508,
 'trial_height': 0.3937,
 'drop_time': 283.31,
 'est_step_off': 6718,
 'max_ch1_rms': 2.4182735804634612e-05,
 'max_ch2_rms': 0.0002964230176802534,
 'max_ch1_rms_row': 7002,
 'max_ch2_rms_row': 6969,
 'ch1_preactivation': 0,
 'ch2_preactivation': 33}